# Datasets

In this tutorial, we will learn how to read Open3D-ML datasets.

You can use any dataset available in the `ml3d.datasets` namespace. For this example, we will use the `SemanticKITTI` dataset. You can use any of the other datasets to load data. However, you must understand that the parameters may vary for each dataset.

To read a dataset in this example, we will supply the following parameter variables:

- Dataset path (`dataset_path`)
- Cache directory (`cache_dir`)
- Dataset splits (for training, validation, and testing)

> **For more theoretical background information on dataset splitting, please refer to these articles:**
>
> https://machinelearningcompass.com/dataset_optimization/split_data_the_right_way/
>
> https://www.freecodecamp.org/news/key-machine-learning-concepts-explained-dataset-splitting-and-random-forest/

## Creating a global dataset object

First, we import the Open3D-ML PyTorch library:

In [2]:
# import torch
import open3d.ml.torch as ml3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


We then create a `dataset` object, initializing it with *path, cache directory*, and *splits*. This `dataset` can read all the files inside `dataset_path` directory:

In [3]:
# Read a dataset by specifying the path. We are also providing the cache directory and splits.
dataset = ml3d.datasets.SemanticKITTI(dataset_path='/root/data/kitti_odometry/',
                                      cache_dir='logs/cache',
                                      training_split=['00'],
                                      validation_split=['01'],
                                      test_split=['01'])

A couple of words regarding the *splits* variables: here, we isolate different portions of the `SemanticKITTI` dataset content and divide them into 3 different parts:

1. `training_split` for data training. This part usually contains 70-75% of the global `dataset` content.
2. `validation_split` for data validation. This part accounts for 10-15% of the global `dataset` content;
3. `test_split` for testing. It contains test data and its size varies.

Note the `SemanticKITTI` dataset folder structure:

![dataset_structure](https://user-images.githubusercontent.com/93158890/162548755-28c541d3-3557-4903-a9a1-cc685d16dfc2.jpg)

The three different *split* parameter variables instruct Open3D-ML subsystem to reference the following folder locations:

- `training_split=['00']` points to `'SemanticKITTI/dataset/sequences/00/'`
- `validation_split=['01']` points to `'SemanticKITTI/dataset/sequences/01/'`
- `test_split=['01']` points to `'SemanticKITTI/dataset/sequences/01/'`

> Note: **dataset split directories usually contain numerous point cloud files.** In our example we included only one point cloud file for extra speed and convenience.

## Creating dataset split objects to query the data

Next, we will create **dedicated** dataset split objects for specifying which split portion we would like to query.

First, we create a `train_split` subset for training from the global `dataset` content we have initialized above using the `get_split()` method:

In [4]:
# Split the dataset for 'training'. You can get the other splits by passing 'validation' or 'test'
train_split = dataset.get_split('training')

INFO - 2022-10-19 03:12:13,650 - semantickitti - Found 4541 pointclouds for training


Now, let's do the same for validation:

In [5]:
# Similarly, get validataion split.
val_split = dataset.get_split('validation')

INFO - 2022-10-19 03:12:13,718 - semantickitti - Found 1101 pointclouds for validation


Finally, we create a `test_split` subset for testing:

In [6]:
# Get test split
test_split = dataset.get_split('test')

INFO - 2022-10-19 03:12:13,782 - semantickitti - Found 1101 pointclouds for test


## Determining the size of dataset splits

Let's see how large out *split* portions are:

In [7]:
# Get length of splits
print(len(train_split))
print(len(val_split))
print(len(test_split))

4541
1101
1101


Above, Open3D-ML prints out the number of pointcloud files it found in `'SemanticKITTI/dataset/sequences/'`' `'/00/'` and `'/01/'` subdirectories we have specified earlier in `training_split=['00'], validation_split=['01'], test_split=['01']` varables for the `dataset`.

## Querying dataset splits for data

In this section, we are using the `train_split` dataset split object as an example. The procedure would be identical for all other dataset splits - `val_split` and `test_split`.

In order to extract the data from the `train_split`, we can iterate through the `train_split` with the index `i` (ranging from `0` - `len(train_split)-1`) using the `get_data()` method.
:

In [8]:
# Query splits for data, index should be from `0` to `len(split) - 1`
for i in range(len(train_split)):
    data = train_split.get_data(i)
    print(data)
    break

{'point': array([[ 5.2897942e+01,  2.2989739e-02,  1.9979945e+00],
       [ 5.3750526e+01,  1.9291429e-01,  2.0269539e+00],
       [ 5.3803116e+01,  3.6183926e-01,  2.0289137e+00],
       ...,
       [ 3.8401384e+00, -1.4381756e+00, -1.7735560e+00],
       [ 3.8257158e+00, -1.4192016e+00, -1.7645701e+00],
       [ 4.0923753e+00, -1.5071962e+00, -1.8955611e+00]], dtype=float32), 'feat': None, 'label': array([0, 0, 0, ..., 9, 9, 9], dtype=int32)}


`data` objects from the above `for` loop return a dictionary of points (`'point'`), features (`'feat'`), and labels (`'label'`), as we will see below:

In [9]:
data = train_split.get_data(0)  # Dictionary of `point`, `feat`, and `label`
print(data.keys())

dict_keys(['point', 'feat', 'label'])


- The **`'point'`** key value contains a set of 3D points/coordinates - X, Y, and Z:

![dataset_coordinates](https://user-images.githubusercontent.com/93158890/162549410-6369cbd0-b835-4216-ba54-945e3f591395.jpg)

- The **`'feat'`** (*features*) key value contains RGB color information for each of the above points.

- The **`'label'`** key value represents which class the dataset content belongs to, i.e.: *pedestrian, vehicle, traffic light*, etc.

### Querying dataset splits for attributes

We can also extract corresponding point cloud information:

In [10]:
attr = train_split.get_attr(0)
print(
    attr
)  # Dictionary containing information about the data e.g. name, path, split, etc.

{'idx': 0, 'name': '00_000000', 'path': '/root/data/kitti_odometry/dataset/sequences/00/velodyne/000000.bin', 'split': 'training'}


Atttributes returned are: `'idx'`(*index*), `'name'`, `'path'`, and `'split'`.

In [11]:
#support of Open3d-ML visualizer in Jupyter Notebooks is in progress
#view the frames using the visualizer
# vis = ml3d.vis.Visualizer()
# vis.visualize_dataset(dataset, 'training',indices=range(len(train_split)))